# 🎯 GovBR Data Lake - Demonstração Completa

## Apresentação da Solução de Data Lake com Arquitetura Medallion

Este notebook demonstra a arquitetura completa do Data Lake GovBR:
- **Bronze**: Dados brutos das APIs
- **Prata**: Dados transformados e relacionados
- **Ouro**: Dados enriquecidos para análise

### Tecnologias Utilizadas
- ✅ Apache Spark 4.0.1
- ✅ Delta Lake (via JARs)
- ✅ MinIO (S3-compatible)
- ✅ Python/PySpark

In [75]:
# ⚠️ IMPORTANTE: Execute primeiro o notebook CONFIGURAR_SPARK.ipynb
# Se ainda não executou, execute o script de correção:
import sys

def verificar_spark_funcional():
    """Verifica se Spark está funcional, não apenas se existe"""
    try:
        spark
    except NameError:
        return False, "Spark não existe"
    
    try:
        # Tentar acessar uma propriedade simples
        _ = spark.sparkContext
        # Tentar criar um DataFrame de teste
        test_df = spark.range(1)
        test_df.collect()
        return True, "Spark funcional"
    except Exception as e:
        return False, f"Spark existe mas não está funcional: {e}"

spark_ok, mensagem = verificar_spark_funcional()

if spark_ok:
    print(f"✅ {mensagem}")
else:
    print(f"⚠️  {mensagem}")
    
    # Verificar se é erro de Connection Refused (sessão zombie)
    if "Connection refused" in str(mensagem) or "Errno 111" in str(mensagem):
        print("\n🔧 Detectado: Sessão Spark 'zombie' (Connection Refused)")
        print("   Recuperando Spark Session...")
        try:
            exec(open('/home/jovyan/work/recuperar_spark.py').read())
            # Verificar novamente
            spark_ok, mensagem = verificar_spark_funcional()
            if spark_ok:
                print("✅ Spark Session recuperada com sucesso!")
            else:
                print(f"❌ Falha na recuperação: {mensagem}")
                print("💡 Execute CONFIGURAR_SPARK.ipynb manualmente")
        except (Exception, SystemExit, RuntimeError) as e:
            print(f"❌ Erro ao recuperar: {e}")
            import traceback
            traceback.print_exc()
            print("💡 Execute CONFIGURAR_SPARK.ipynb manualmente")
    else:
        print("\n🔧 Tentando configurar Spark automaticamente...")
        try:
            # Tentar versão V2 primeiro (mais robusta)
            try:
                exec(open('/home/jovyan/work/fix_spark_py4j_v2.py').read())
            except:
                # Fallback para versão original
                exec(open('/home/jovyan/work/fix_spark_py4j.py').read())
            # Verificar novamente
            spark_ok, mensagem = verificar_spark_funcional()
            if spark_ok:
                print("✅ Spark configurado com sucesso!")
            else:
                print(f"❌ Falha ao configurar: {mensagem}")
                print("💡 Execute CONFIGURAR_SPARK.ipynb manualmente")
        except (Exception, SystemExit, RuntimeError) as e:
            print(f"❌ Erro ao executar script de correção: {e}")
            import traceback
            traceback.print_exc()
            print("💡 Execute manualmente: CONFIGURAR_SPARK.ipynb")

✅ Spark funcional


## 📊 Visão Geral da Arquitetura

In [76]:
print("=" * 80)
print("GOVBR DATA LAKE - ARQUITETURA MEDALLION")
print("=" * 80)

# Função auxiliar para verificar Spark de forma segura
def verificar_spark_info():
    """Verifica informações do Spark com tratamento robusto de erros"""
    info = {}
    
    # Tentar obter versão
    try:
        info['version'] = spark.version
    except Exception as e:
        try:
            # Método alternativo via SparkContext
            info['version'] = spark.sparkContext.version
        except Exception as e2:
            print(f"⚠️  Não foi possível obter versão: {e2}")
            info['version'] = "N/A (erro Py4J)"
    
    # Tentar obter app name
    try:
        info['app_name'] = spark.sparkContext.appName
    except Exception as e:
        print(f"⚠️  Não foi possível obter appName: {e}")
        info['app_name'] = "N/A"
    
    # Tentar obter master
    try:
        info['master'] = spark.sparkContext.master
    except Exception as e:
        print(f"⚠️  Não foi possível obter master: {e}")
        info['master'] = "N/A"
    
    # Teste básico de funcionalidade
    try:
        test_df = spark.range(1)
        test_df.collect()
        info['funcional'] = True
    except Exception as e:
        print(f"⚠️  Spark não está totalmente funcional: {e}")
        info['funcional'] = False
    
    return info

# Verificar Spark com tratamento de erros robusto
try:
    spark_info = verificar_spark_info()
    
    print(f"\n✅ Spark versão: {spark_info.get('version', 'N/A')}")
    print(f"✅ App: {spark_info.get('app_name', 'N/A')}")
    print(f"✅ Master: {spark_info.get('master', 'N/A')}")
    
    if not spark_info.get('funcional', False):
        print("\n⚠️  ATENÇÃO: Spark Session existe mas pode não estar totalmente funcional")
        print("💡 Se houver erros, execute CONFIGURAR_SPARK.ipynb novamente")
    
except Exception as e:
    print(f"\n❌ Erro ao verificar Spark Session: {e}")
    print("\n💡 SOLUÇÕES:")
    print("1. Execute CONFIGURAR_SPARK.ipynb novamente")
    print("2. Verifique processos Java: !ps aux | grep java")
    print("3. Reinicie o container: docker restart govbr-jupyter-delta")
    print("4. Execute diagnóstico: exec(open('/home/jovyan/work/diagnostico_spark.py').read())")
    raise

print("\n" + "=" * 80)

GOVBR DATA LAKE - ARQUITETURA MEDALLION

✅ Spark versão: 3.5.0
✅ App: GovBR Data Lake - JARs Manuais
✅ Master: local[1]



## 🥉 Camada Bronze - Dados Brutos

### ⚠️ Se os dados Bronze não aparecerem (erro S3A)

Execute esta célula para reiniciar o Spark com os JARs corretos:

In [77]:
# Reiniciar Spark com JARs do Hadoop AWS
# Execute esta célula se houver erro "ClassNotFoundException: S3AFileSystem"

print("🔄 Reiniciando Spark com JARs do Hadoop AWS...")
print("=" * 80)

try:
    exec(open('/home/jovyan/work/spark_com_jars_manual.py').read())
    print("\n✅ Spark reiniciado com sucesso!")
    print("💡 Execute novamente a célula acima para ver os dados Bronze")
except Exception as e:
    print(f"\n❌ Erro ao reiniciar Spark: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Tente executar manualmente:")
    print("   exec(open('/home/jovyan/work/spark_com_jars_manual.py').read())")

🔄 Reiniciando Spark com JARs do Hadoop AWS...
🚀 INICIALIZANDO SPARK COM JARs MANUAIS

[1/4] Baixando JARs necessários...
  ✅ hadoop-aws-3.3.4.jar já existe
  ✅ aws-java-sdk-bundle-1.12.262.jar já existe
  ✅ 2 JARs prontos

[2/4] Limpando Spark existente...
  ✅ Spark limpo

[3/4] Matando processos Java...

[4/4] Inicializando Spark com JARs...
  ℹ️  Criando SparkSession com JARs locais...
     JARs: /tmp/spark_jars/hadoop-aws-3.3.4.jar,/tmp/spark_jars/aws-java-sdk-bundle-1.12.262.jar
  ℹ️  Inicializando Spark (aguarde alguns segundos)...
  ℹ️  Configurando S3A...
  ℹ️  Verificando funcionalidade...
  ✅ Spark inicializado com sucesso!
     Versão: 3.5.0
     App: GovBR Data Lake - JARs Manuais
  ℹ️  Testando acesso S3A...
  ✅ S3A configurado - tentando ler dados...
  ✅ S3A funcionando! Encontrados 5,571 registros em municipios

✅ SPARK INICIALIZADO COM SUCESSO!

Agora você pode ler os dados Bronze:
  df = spark.read.parquet('s3a://govbr/bronze/ibge/municipios/')
  df.show()

✅ Spark rein

## ⚠️ Executar Ingestão Bronze

Se a camada Bronze estiver vazia, execute a célula abaixo para fazer a ingestão dos dados:

In [79]:
# Executar ingestão Bronze
# ⚠️ Execute esta célula apenas se a camada Bronze estiver vazia

print("🥉 Executando ingestão Bronze...")
print("=" * 80)

try:
    # Executar script de ingestão
    exec(open('/home/jovyan/work/01_bronze_ingestion.py').read())
    
    print("\n✅ Ingestão Bronze concluída!")
    print("💡 Execute novamente a célula acima para ver os dados Bronze gerados")
    
except Exception as e:
    print(f"\n❌ Erro ao executar ingestão: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Verifique:")
    print("   1. Conexão com a internet (APIs externas)")
    print("   2. Conexão com MinIO")
    print("   3. Permissões de escrita no bucket")

🥉 Executando ingestão Bronze...
CAMADA BRONZE - INGESTÃO DE DADOS BRUTOS

[1/5] Coletando municípios do Brasil (IBGE)...
✅ Bronze: bronze/ibge/municipios/dt=20251114/data.parquet (5571 registros, 132.35 KB)

[2/5] Coletando estados do Brasil (IBGE)...
✅ Bronze: bronze/ibge/estados/dt=20251114/data.parquet (27 registros, 4.86 KB)

[3/5] Coletando órgãos SIAFI...
✅ Bronze: bronze/portal_transparencia/orgaos_siafi/dt=20251114/data.parquet (6 registros, 2.10 KB)

[4/5] Coletando dados de BPC (amostra SP - primeiros 50 municípios)...
  Progresso: 3270/50 municípios...
  Progresso: 3280/50 municípios...
  Progresso: 3290/50 municípios...
  Progresso: 3300/50 municípios...
  Progresso: 3310/50 municípios...
✅ Bronze: bronze/portal_transparencia/bpc_municipios/dt=20251114/data.parquet (50 registros, 10.19 KB)
  ✅ 50 municípios com dados de BPC

[4b/5] Coletando dados de Bolsa Família (amostra SP - primeiros 50 municípios)...
  Progresso: 3270/50 municípios...
  Progresso: 3280/50 municípios...

## ⚠️ Gerar Dados Prata

Se a camada Prata não tiver dados, execute a célula abaixo para gerar:

In [80]:
# Executar transformação Prata
# ⚠️ Execute esta célula apenas se a camada Prata estiver vazia

print("🔄 Executando transformação Prata...")
print("=" * 80)

try:
    # Verificar se Spark está funcional
    test_df = spark.range(1)
    test_df.collect()
    
    # Executar script de transformação
    exec(open('/home/jovyan/work/02_prata_transformacao.py').read())
    
    print("\n✅ Transformação Prata concluída!")
    print("💡 Execute novamente a célula acima para ver os dados Prata gerados")
    
except Exception as e:
    print(f"\n❌ Erro ao executar transformação: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Verifique se os dados Bronze estão disponíveis")

🔄 Executando transformação Prata...
CAMADA PRATA - TRANSFORMAÇÃO E RELACIONAMENTO

[1/4] Carregando dados da camada Bronze...
✅ Lido Bronze: bronze/ibge/municipios/dt=20251114/data.parquet (5571 registros)
✅ Lido Bronze: bronze/ibge/estados/dt=20251114/data.parquet (27 registros)
✅ Lido Bronze: bronze/portal_transparencia/orgaos_siafi/dt=20251114/data.parquet (6 registros)
✅ Lido Bronze: bronze/portal_transparencia/bpc_municipios/dt=20251114/data.parquet (50 registros)
✅ Lido Bronze: bronze/ibge/populacao_estados/dt=20251114/data.parquet (27 registros)

📊 Status dos dados Bronze carregados:
  - Municípios: ✅ (5571 registros)
  - Estados: ✅ (27 registros)
  - Órgãos: ✅ (6 registros)
  - BPC: ✅ (50 registros)
  - Bolsa Família: ❌ (0 registros)
  - População: ✅ (27 registros)

[2/4] Tratando e limpando dados...

[3/4] Criando dimensões e relacionamentos...
  ✅ População adicionada aos municípios

[Processando BPC] 50 registros encontrados
  ✅ Merge com dim_municipios concluído (50 registr

In [81]:
# Listar dados Bronze
# O Spark lê automaticamente partições quando apontamos para o diretório pai
bronze_datasets = [
    "s3a://govbr/bronze/ibge/municipios/",
    "s3a://govbr/bronze/ibge/estados/",
    "s3a://govbr/bronze/ibge/populacao_estados/",
    "s3a://govbr/bronze/portal_transparencia/orgaos_siafi/",
    "s3a://govbr/bronze/portal_transparencia/bpc_municipios/"
]

print("🥉 CAMADA BRONZE - Dados Brutos\n")
bronze_summary = []

for path in bronze_datasets:
    try:
        # Tentar ler - Spark deve detectar partições automaticamente
        df = spark.read.parquet(path)
        count = df.count()
        dataset_name = path.split('/')[-2]
        bronze_summary.append({
            'dataset': dataset_name,
            'registros': count,
            'colunas': len(df.columns)
        })
        print(f"✅ {dataset_name:30s} | {count:>10,} registros | {len(df.columns):>3} colunas")
    except Exception as e:
        dataset_name = path.split('/')[-2]
        error_msg = str(e)
        # Se for erro de classe não encontrada, é problema de configuração S3A
        if "ClassNotFoundException" in error_msg or "S3AFileSystem" in error_msg:
            print(f"⚠️  {dataset_name:30s} | Erro S3A - Execute: exec(open('/home/jovyan/work/inicializar_spark.py').read())")
        elif "Path does not exist" in error_msg or "does not exist" in error_msg:
            print(f"⚠️  {dataset_name:30s} | Não disponível")
        else:
            print(f"⚠️  {dataset_name:30s} | Erro: {error_msg[:50]}...")

print(f"\n📊 Total de datasets Bronze: {len(bronze_summary)}")

if len(bronze_summary) == 0:
    print("\n💡 DICAS:")
    print("   1. Verifique se a ingestão foi executada: exec(open('/home/jovyan/work/verificar_bronze.py').read())")
    print("   2. Se os dados existem mas não aparecem, reinicie o Spark: exec(open('/home/jovyan/work/inicializar_spark.py').read())")
    print("   3. Execute a ingestão: exec(open('/home/jovyan/work/01_bronze_ingestion.py').read())")

🥉 CAMADA BRONZE - Dados Brutos

✅ municipios                     |      5,571 registros |  11 colunas
✅ estados                        |         27 registros |   7 colunas
✅ populacao_estados              |         27 registros |   5 colunas
✅ orgaos_siafi                   |          6 registros |   3 colunas
✅ bpc_municipios                 |         50 registros |  13 colunas

📊 Total de datasets Bronze: 5


## 🔄 Camada Prata - Dados Transformados

## ⚠️ Gerar Dados Ouro

Se a camada Ouro não tiver dados, execute a célula abaixo para gerar:

In [82]:
# Executar enriquecimento Ouro
# ⚠️ Execute esta célula apenas se a camada Ouro estiver vazia
# ⚠️ IMPORTANTE: A camada Prata deve estar gerada primeiro!

print("🏆 Executando enriquecimento Ouro...")
print("=" * 80)

try:
    # Verificar se Spark está funcional
    test_df = spark.range(1)
    test_df.collect()
    
    # Verificar se Prata existe
    try:
        prata_test = spark.read.parquet("s3a://govbr/prata/dim_municipios/")
        prata_count = prata_test.count()
        print(f"✅ Camada Prata disponível ({prata_count:,} registros em dim_municipios)")
    except Exception as e:
        print("❌ Camada Prata não está disponível!")
        print("💡 Execute primeiro a transformação Prata (célula acima)")
        raise
    
    # Executar script de enriquecimento
    exec(open('/home/jovyan/work/03_ouro_enriquecimento.py').read())
    
    print("\n✅ Enriquecimento Ouro concluído!")
    print("💡 Execute novamente a célula acima para ver os dados Ouro gerados")
    
except Exception as e:
    print(f"\n❌ Erro ao executar enriquecimento: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Verifique se:")
    print("   1. A camada Prata está gerada")
    print("   2. Os dados Bronze estão disponíveis")
    print("   3. Spark está funcionando corretamente")

🏆 Executando enriquecimento Ouro...
✅ Camada Prata disponível (5,571 registros em dim_municipios)
CAMADA OURO - ENRIQUECIMENTO E DADOS FINAIS

[1/5] Carregando dados da camada Prata...
✅ Lido Prata: prata/dim_municipios/dt=20251114/data.parquet (5571 registros)
✅ Lido Prata: prata/dim_estados/dt=20251114/data.parquet (27 registros)
✅ Lido Prata: prata/fato_bpc/dt=20251114/data.parquet (50 registros)
✅ Lido Prata: prata/dim_orgaos/dt=20251114/data.parquet (6 registros)

[2/5] Enriquecendo dimensão de municípios...
✅ Ouro: ouro/dim_municipios_enriquecida/dt=20251114/data.parquet (5571 registros, 142.47 KB)

[3/5] Enriquecendo dimensão de estados...
✅ Ouro: ouro/dim_estados_enriquecida/dt=20251114/data.parquet (27 registros, 13.78 KB)

[4/5] Criando fato BPC enriquecido...
✅ Ouro: ouro/fato_bpc_enriquecido/dt=20251114/data.parquet (50 registros, 19.49 KB)

[5/5] Criando tabelas agregadas para análise...
✅ Ouro: ouro/agregacao_bpc_por_regiao/dt=20251114/data.parquet (1 registros, 6.78 KB)


In [83]:
# Listar dados Prata
prata_datasets = [
    "s3a://govbr/prata/dim_municipios/",
    "s3a://govbr/prata/dim_estados/",
    "s3a://govbr/prata/dim_orgaos/",
    "s3a://govbr/prata/fato_bpc/",
    "s3a://govbr/prata/fato_bolsa_familia/"
]

print("🔄 CAMADA PRATA - Dados Transformados\n")
prata_summary = []

for path in prata_datasets:
    try:
        df = spark.read.parquet(path)
        count = df.count()
        dataset_name = path.split('/')[-2]
        prata_summary.append({
            'dataset': dataset_name,
            'registros': count,
            'colunas': len(df.columns)
        })
        print(f"✅ {dataset_name:30s} | {count:>10,} registros | {len(df.columns):>3} colunas")
    except Exception as e:
        print(f"⚠️  {path.split('/')[-2]:30s} | Não disponível")

print(f"\n📊 Total de datasets Prata: {len(prata_summary)}")

🔄 CAMADA PRATA - Dados Transformados

✅ dim_municipios                 |      5,571 registros |  16 colunas
✅ dim_estados                    |         27 registros |  13 colunas
✅ dim_orgaos                     |          6 registros |   3 colunas
⚠️  fato_bpc                       | Não disponível
⚠️  fato_bolsa_familia             | Não disponível

📊 Total de datasets Prata: 3


## 🏆 Camada Ouro - Dados Enriquecidos

In [84]:
# Listar dados Ouro
ouro_datasets = [
    "s3a://govbr/ouro/municipios_enriquecidos/",
    "s3a://govbr/ouro/estados_enriquecidos/",
    "s3a://govbr/ouro/bpc_analytics/",
    "s3a://govbr/ouro/rankings/",
    "s3a://govbr/ouro/agregacoes_regionais/"
]

print("🏆 CAMADA OURO - Dados Enriquecidos\n")
ouro_summary = []

for path in ouro_datasets:
    try:
        df = spark.read.parquet(path)
        count = df.count()
        dataset_name = path.split('/')[-2]
        ouro_summary.append({
            'dataset': dataset_name,
            'registros': count,
            'colunas': len(df.columns)
        })
        print(f"✅ {dataset_name:30s} | {count:>10,} registros | {len(df.columns):>3} colunas")
    except Exception as e:
        print(f"⚠️  {path.split('/')[-2]:30s} | Não disponível")

print(f"\n📊 Total de datasets Ouro: {len(ouro_summary)}")

🏆 CAMADA OURO - Dados Enriquecidos

⚠️  municipios_enriquecidos        | Não disponível
⚠️  estados_enriquecidos           | Não disponível
⚠️  bpc_analytics                  | Não disponível
⚠️  rankings                       | Não disponível
⚠️  agregacoes_regionais           | Não disponível

📊 Total de datasets Ouro: 0


## 📈 Análise Demonstrativa

In [85]:
# Exemplo: Análise cruzando as três camadas
try:
    # Ler dados de cada camada
    df_bronze_municipios = spark.read.parquet("s3a://govbr/bronze/ibge/municipios/")
    
    print("📊 Exemplo de Análise: Municípios por Região\n")
    
    from pyspark.sql.functions import count, col
    
    resultado = df_bronze_municipios \
        .groupBy("regiao_nome") \
        .agg(count("*").alias("total_municipios")) \
        .orderBy(col("total_municipios").desc())
    
    resultado.show(truncate=False)
    
except Exception as e:
    print(f"⚠️  Erro na análise: {e}")

📊 Exemplo de Análise: Municípios por Região

+------------+----------------+
|regiao_nome |total_municipios|
+------------+----------------+
|Nordeste    |1794            |
|Sudeste     |1668            |
|Sul         |1191            |
|Centro-Oeste|467             |
|Norte       |450             |
|NULL        |1               |
+------------+----------------+



## ✅ Resumo da Arquitetura

### Fluxo de Dados:
```
APIs → Bronze → Prata → Ouro
  ↓       ↓       ↓       ↓
IBGE   Parquet  Dimensões  Métricas
Portal          Fatos      Rankings
```

### Benefícios:
- ✅ **Bronze**: Preserva dados originais
- ✅ **Prata**: Dados limpos e relacionados
- ✅ **Ouro**: Pronto para análise e dashboards
- ✅ **Escalável**: Spark processa grandes volumes
- ✅ **Flexível**: MinIO armazena em formato Parquet

### Tecnologias:
- Apache Spark 4.0.1
- Delta Lake (via JARs)
- MinIO (S3-compatible)
- Python/PySpark